In [1]:
import numpy as np
from descent_env import DescentEnv
import random 

pygame 2.5.2 (SDL 2.28.3, Python 3.10.0)
Hello from the pygame community. https://www.pygame.org/contribute.html
Using Python-based geo functions


In [2]:
# Cambiar render_mode a rgb_array para entrenar/testear
env = DescentEnv(render_mode='rgb_array')

Reading config from C:\Users\fbrio\bluesky\settings.cfg
Reading magnetic variation data
Loading global navigation database...
Reading cache: C:\Users\fbrio\bluesky\cache\navdata.p
Successfully loaded OpenAP performance model
Failed to load BADA performance model
Successfully loaded legacy performance model
Successfully loaded plugin AREA
Successfully loaded plugin DATAFEED


In [4]:
env.observation_space
env.action_space

Box(-1.0, 1.0, (1,), float64)

In [8]:
altitude_space = np.linspace(-2, 2, 100)
vertical_velocity_space =  np.linspace(-3, 3, 100)
target_altitude_space =  np.linspace(-2, 2, 100)
runway_distance_space = np.linspace(-2, 2, 100)
altitude_space

array([-2.        , -1.95959596, -1.91919192, -1.87878788, -1.83838384,
       -1.7979798 , -1.75757576, -1.71717172, -1.67676768, -1.63636364,
       -1.5959596 , -1.55555556, -1.51515152, -1.47474747, -1.43434343,
       -1.39393939, -1.35353535, -1.31313131, -1.27272727, -1.23232323,
       -1.19191919, -1.15151515, -1.11111111, -1.07070707, -1.03030303,
       -0.98989899, -0.94949495, -0.90909091, -0.86868687, -0.82828283,
       -0.78787879, -0.74747475, -0.70707071, -0.66666667, -0.62626263,
       -0.58585859, -0.54545455, -0.50505051, -0.46464646, -0.42424242,
       -0.38383838, -0.34343434, -0.3030303 , -0.26262626, -0.22222222,
       -0.18181818, -0.14141414, -0.1010101 , -0.06060606, -0.02020202,
        0.02020202,  0.06060606,  0.1010101 ,  0.14141414,  0.18181818,
        0.22222222,  0.26262626,  0.3030303 ,  0.34343434,  0.38383838,
        0.42424242,  0.46464646,  0.50505051,  0.54545455,  0.58585859,
        0.62626263,  0.66666667,  0.70707071,  0.74747475,  0.78

In [9]:
def get_state(obs):
    alt = obs['altitude'][0]
    vz = obs['vz'][0]
    target_alt = obs['target_altitude'][0]
    runway_dist = obs['runway_distance'][0]
    alt_idx = min(np.digitize(alt, altitude_space), len(altitude_space)-1)
    vz_idx = min(np.digitize(vz, vertical_velocity_space), len(vertical_velocity_space)-1)
    target_alt_idx = min(np.digitize(target_alt, target_altitude_space), len(target_altitude_space)-1)
    runway_dist_idx = min(np.digitize(runway_dist, runway_distance_space), len(runway_distance_space)-1)
    return alt_idx, vz_idx, target_alt_idx, runway_dist_idx

In [10]:
obs = env.observation_space.sample()
print(obs)
state = get_state(obs) # Ejemplo de obs
state

OrderedDict([('altitude', array([-2.40008052])), ('runway_distance', array([0.43023137])), ('target_altitude', array([-0.77193812])), ('vz', array([-0.34500857]))])


(0, 44, 31, 61)

In [11]:
actions = list(np.linspace(-1, 1, 10))
actions

[-1.0,
 -0.7777777777777778,
 -0.5555555555555556,
 -0.33333333333333337,
 -0.11111111111111116,
 0.11111111111111116,
 0.33333333333333326,
 0.5555555555555554,
 0.7777777777777777,
 1.0]

In [12]:
def get_sample_action():
    return random.choice(actions)

In [13]:
Q = np.zeros((len(altitude_space), len(vertical_velocity_space), len(target_altitude_space), len(runway_distance_space), len(actions)))
Q

array([[[[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],

         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],

         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],

         ...,

         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0.

In [14]:
# Tamaños de cada dimensión del estado
n_alt = len(altitude_space)
n_vz = len(vertical_velocity_space)
n_target_alt = len(target_altitude_space)
n_rwy_dist = len(runway_distance_space)
n_actions = len(actions)

# Inicializamos Q-table con ceros
Q = np.zeros((n_alt, n_vz, n_target_alt, n_rwy_dist, n_actions))


In [15]:
def choose_action(state, epsilon):
    if np.random.random() < epsilon:
        return np.random.randint(n_actions)  # acción aleatoria
    else:
        return np.argmax(Q[state])  # mejor acción según Q


In [25]:
# Hiperparámetros
alphas = [0.8] # Tasa de aprendizaje
gammas = [0.4] # Factor de descuento
episodes = 10000 # Episodios de entrenamiento
test_episodes = 100  # Episodios de testeo

epsilon = 1  # Valor inicial de epsilon
epsilon_decay = 0.9999  # Decaimiento de epsilon
epsilon_min = 0.01  # Valor mínimo de epsilon

In [21]:
def train(alpha, gamma, epsilon, epsilon_min, epsilon_decay, episodes):
    Q = np.zeros((n_alt, n_vz, n_target_alt, n_rwy_dist, n_actions))
    rewards_per_episode = []

    for ep in range(episodes):
        obs, _ = env.reset()
        state = get_state(obs)
        done = False
        total_reward = 0

        while not done:
            action_idx = choose_action(state, epsilon)
            action = np.array([actions[action_idx]])

            next_obs, reward, terminated, truncated, _ = env.step(action)
            next_state = get_state(next_obs)
            done = terminated or truncated

            best_next = np.argmax(Q[next_state])
            td_target = reward + gamma * Q[next_state + (best_next,)]
            td_error = td_target - Q[state + (action_idx,)]
            Q[state + (action_idx,)] += alpha * td_error

            state = next_state
            total_reward += reward

        rewards_per_episode.append(total_reward)
        epsilon = max(epsilon * epsilon_decay, epsilon_min)

    return Q, np.mean(rewards_per_episode)


In [22]:
def test_agent(Q):
    rewards = []

    for _ in range(test_episodes):
        obs, _ = env.reset()
        state = get_state(obs)
        done = False
        total_reward = 0

        while not done:
            action_idx = np.argmax(Q[state])  # greedy
            action = np.array([actions[action_idx]])
            next_obs, reward, terminated, _, _ = env.step(action)
            state = get_state(next_obs)
            done = terminated
            total_reward += reward

        rewards.append(total_reward)

    return np.mean(rewards)

In [23]:
results = []

In [26]:
# Entrenamiento y testing por cada epsilon
print(f"\n=== Entrenando con epsilon: {epsilon:.2f} ===")
print(f"\n--- Entrenando con {episodes} episodios ---")
for alpha in alphas:
    for gamma in gammas:
        print(f"\n--- Entrenando con alpha: {alpha:.2f}, gamma: {gamma:.2f} ---")
        Q, train_avg_reward = train(alpha, gamma, epsilon, epsilon_min, epsilon_decay, episodes)
        print(f"--- Entrenamiento finalizado con epsilon {epsilon:.2f} ---")    
        print(f"--- Evaluando desempeño (testeo sin exploración) ---")
        test_avg_reward = test_agent(Q)
        results.append({
        "alpha": alpha,
        "gamma": gamma,
        "episodes": episodes, 
        "train_avg_reward": train_avg_reward,
        "test_avg_reward": test_avg_reward
        })
        with open("resultados_qlearning.txt", "w") as f:
            for res in results:
                f.write(str(res) + "\n")
                print(f"Resultados: {res}")


=== Entrenando con epsilon: 1.00 ===

--- Entrenando con 10000 episodios ---

--- Entrenando con alpha: 0.80, gamma: 0.40 ---
--- Entrenamiento finalizado con epsilon 1.00 ---
--- Evaluando desempeño (testeo sin exploración) ---
Resultados: {'alpha': 0.4, 'gamma': 0.4, 'episodes': 10000, 'train_avg_reward': -135.17822954767368, 'test_avg_reward': -128.19394116225}
Resultados: {'alpha': 0.8, 'gamma': 0.4, 'episodes': 10000, 'train_avg_reward': -136.88093969570102, 'test_avg_reward': -126.09349316326667}
